---

# Books Stratup - Value Proposition
Guy Zamir, April 2021
## Goal of the study
Generate a value proposition for a new product for a startup company within the books field.


## How are we going to achieve it?
1. [Study the tables](#1) <br>
2. [Find the number of books released after January 1, 2000](#2)<br>
3. [Find the number of user reviews and the average rating for each book.](#3)<br>
4. [Identify the publisher that has released the greatest number of books with more than 50 pages](#4)<br>
5. [Identify the author with the highest average book rating: look only at books with at least 50 ratings.](#5)<br>
6. [Find the average number of text reviews among users who rated more than 50 books.](#6)<br>
7. [Final Conclusions](#7)<br>

### Study the following tables:
<a id="1"></a>
    - books
    - authors
    - publishers 
    - ratings 
    - reviews 

In [2]:
#!pip install psycopg2-binary --user

In [3]:
pip install psycopg2-binary

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine


db_config = {'user': 'practicum_student',         # username
             'pwd': 's65BlTKV3faNIGhmvJVzOqhs', # password
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # connection port
             'db': 'data-analyst-final-project-db'}          # the name of the database

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})


In [5]:
books = pd.io.sql.read_sql('books', con = engine)
display(books.head())

authors = pd.io.sql.read_sql('authors', con = engine)
display(authors.head())

publishers = pd.io.sql.read_sql('publishers', con = engine)
display(publishers.head())

ratings = pd.io.sql.read_sql('ratings', con = engine)
display(ratings.head())

reviews = pd.io.sql.read_sql('reviews', con = engine)
display(reviews.head())

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [6]:
# query = '''SELECT * from books'''
# pd.io.sql.read_sql(query, con = engine)

def result(query):
    return pd.io.sql.read_sql(query, con = engine)
    
# test = '''SELECT * from books'''

# result(test)

### Find the number of books released after January 1, 2000.
<a id="2"></a>



In [7]:
query = ''' 
SELECT COUNT(book_id) AS number_of_books_released_after_2000
FROM books  
WHERE publication_date > '2000-01-01'
''' 
result(query)

,number_of_books_released_after_2000
0,819


There are 819 books that have been released after January 1,2000 in our database. 

### Find the number of user reviews and the average rating for each book.
<a id="3"></a>

In [9]:
query = ''' 
SELECT 
books.book_id, 
books.title, 
COUNT (DISTINCT reviews.review_id) AS count_of_user_reviews,
AVG(ratings.rating) AS avg_book_rating

FROM books AS books

LEFT OUTER JOIN reviews AS reviews ON books.book_id = reviews.book_id 
LEFT OUTER JOIN ratings AS ratings ON books.book_id = ratings.book_id 

GROUP BY books.book_id
ORDER BY count_of_user_reviews DESC;
'''

result(query)

,book_id,title,count_of_user_reviews,avg_book_rating
0,948,Twilight (Twilight #1),7,3.662500
1,963,Water for Elephants,6,3.977273
2,734,The Glass Castle,6,4.206897
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
4,695,The Curious Incident of the Dog in the Night-Time,6,4.081081
...,...,...,...,...
995,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667
996,808,The Natural Way to Draw,0,3.000000
997,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000000
998,221,Essential Tales and Poems,0,4.000000


### Identify the publisher that has released the greatest number of books with more than 50 pages 
<a id="4"></a>
This helps to exclude brochures and similar publications from the analysis.

In [59]:
query = '''
SELECT 
    publishers.publisher_id,
    publishers.publisher,
    COUNT (books.book_id) AS num_of_books_over_50_pages
FROM
    publishers
LEFT OUTER JOIN books AS books ON publishers.publisher_id = books.publisher_id
WHERE books.num_pages > 50
GROUP BY 
    publishers.publisher_id
ORDER BY
    num_of_books_over_50_pages DESC
LIMIT 1;
'''

result(query)

,publisher_id,publisher,num_of_books_over_50_pages
0,212,Penguin Books,42


Penguin Books is the one publisher that has the highest number of books with over 50 pages 

### Identify the author with the highest average book rating: look only at books with at least 50 ratings.
<a id="5"></a>

In [76]:
query = '''
SELECT 
    books.book_id, 
    AVG(ratings.rating) AS avg_book_rating,
    COUNT (DISTINCT ratings.rating_id) AS number_of_ratings,
    authors.author AS author    
FROM
    books AS books
LEFT JOIN ratings AS ratings ON books.book_id = ratings.book_id 
LEFT JOIN authors AS authors ON books.author_id = authors.author_id  

GROUP BY books.book_id, author
HAVING COUNT (DISTINCT ratings.rating_id) >= 50
ORDER BY AVG(ratings.rating) DESC
LIMIT 1;
'''

result(query)

,book_id,avg_book_rating,number_of_ratings,author
0,302,4.414634,82,J.K. Rowling/Mary GrandPré


J.K. Rowling/Mary GrandPré is the author with the highest average book rating among the books with over 50 ratings 

### Find the average number of text reviews among users who rated more than 50 books.
<a id="6"></a>

In [32]:
query = '''
SELECT 
    AVG(Sub.count_of_reviews) AS avg_count_of_reviews
FROM
    (SELECT 
    username, 
    COUNT(review_id) AS count_of_reviews
FROM 
    reviews
WHERE 
    username IN 
    (SELECT 
        username
    FROM
        ratings
    GROUP BY
        username
    HAVING
        COUNT(rating_id) > 50)
GROUP BY username) AS Sub;
'''
result(query)

,avg_count_of_reviews
0,24.333333


The avg. number of text reviews for users who rated more than 50 books is 24.3, that means that heavy users typically don't go much above 25ish ratings

### Final conclusions
<a id="7"></a>

A few final conclusion to sum up our learnings:
- examining existing database of other player within this field, it is evident that over 80% of their books were published post the year 2000. It is now worthwhile to further examine whether this is the book lovers preferences / market or could we potentially diffrentiate ourselves by looking at older classics which are less popular at our competitors service?

- we can see that many books have multiple reviews while others does not have reviews at all. Again, can we take advantage and make sure to generate some offering for books that do not have reviews at all? or can we create some other way to diffrentiate ourselves and create a better review type focused on the top most popular ones?

- Penguin Books are the most popular publisher and it would be reasonable to either consider to partner with them or make additional research to see what is it that they are doing so good. 

- Same as with the author J.K. Rowling/Mary GrandPr - is there anything we can learn from her popularity? is it the genre or is it the writing style of this specific persona? potentially, we could consider some collboration with her.
- If we choose to monitise our business through reviewers, it is evident that the heavy ones do not typically send much more than 25 reviews. Should we plan and prioritise our compensation plan to reviews with this as a ceiling number. (e.g. gradually increase compensation by 10% for every 5 reviews until ceiling of 25 reviews).